In [4]:
# register model

import json
import mlflow
import logging
import os
import dagshub
from dotenv import load_dotenv
load_dotenv() 
# Set up DagsHub credentials for MLflow tracking
dagshub_token = os.getenv("DAGSHUB_PAT")
if not dagshub_token:
    raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

mlflow.set_tracking_uri(
    "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
)

# logging configuration
logger = logging.getLogger('model_registration')
logger.setLevel('DEBUG')

console_handler = logging.StreamHandler()
console_handler.setLevel('DEBUG')

file_handler = logging.FileHandler('model_registration_errors.log')
file_handler.setLevel('ERROR')

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

def load_model_info(file_path: str) -> dict:
    """Load the model info from a JSON file."""
    try:
        with open(file_path, 'r') as file:
            model_info = json.load(file)
        logger.debug('Model info loaded from %s', file_path)
        return model_info
    except FileNotFoundError:
        logger.error('File not found: %s', file_path)
        raise
    except Exception as e:
        logger.error('Unexpected error occurred while loading the model info: %s', e)
        raise

def register_model(model_name: str, model_info: dict):
    """Register the model to the MLflow Model Registry."""
    try:
        model_uri = f"runs:/{model_info['run_id']}/{model_info['model_path']}"
        
        # Register the model
        model_version = mlflow.register_model(model_uri, model_name)
        
        # Transition the model to "Staging" stage
        client = mlflow.tracking.MlflowClient()
        client.transition_model_version_stage(
            name=model_name,
            version=model_version.version,
            stage="Staging"
        )
        
        logger.debug(f'Model {model_name} version {model_version.version} registered and transitioned to Staging.')
    except Exception as e:
        logger.error('Error during model registration: %s', e)
        raise

def main():
    try:
        model_info_path = r'D:\yt-comment-sentiment-analysis2\experiment_info.json'
        model_info = load_model_info(model_info_path)
        
        model_name = "testing-light-gbm-v2"
        register_model(model_name, model_info)
    except Exception as e:
        logger.error('Failed to complete the model registration process: %s', e)
        print(f"Error: {e}")

if __name__ == '__main__':
    main()

2026-01-29 11:18:22,216 - model_registration - DEBUG - Model info loaded from D:\yt-comment-sentiment-analysis2\experiment_info.json
2026-01-29 11:18:22,216 - model_registration - DEBUG - Model info loaded from D:\yt-comment-sentiment-analysis2\experiment_info.json
2026-01-29 11:18:22,216 - model_registration - DEBUG - Model info loaded from D:\yt-comment-sentiment-analysis2\experiment_info.json
2026-01-29 11:18:22,216 - model_registration - DEBUG - Model info loaded from D:\yt-comment-sentiment-analysis2\experiment_info.json
Successfully registered model 'testing-light-gbm-v2'.
2026-01-29 11:18:24,904 - model_registration - ERROR - Error during model registration: Unable to find a logged_model with artifact_path lgbm_model under run a8baff5083c545c483156fb228d0518a
2026-01-29 11:18:24,904 - model_registration - ERROR - Error during model registration: Unable to find a logged_model with artifact_path lgbm_model under run a8baff5083c545c483156fb228d0518a
2026-01-29 11:18:24,904 - model_

Error: Unable to find a logged_model with artifact_path lgbm_model under run a8baff5083c545c483156fb228d0518a
